# Using Langchain to generate LLM chains

In this notebook, we use Langchain to generate LLM chains.   
The goal is to process the input information in several individual steps.  
Such processing pipeline reduces the hallucination of LLMs and also lead to more precise and controlable performance.   

**We'll cover the following:**
- Sequential Chain
- Router Chain

**Use case:**
- Patients inquiry analysis
- Patient symptoms categorization


In [1]:
import os
import pprint

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [2]:
# A function for printing nicely
def nprint(text, indent=2):
    pp = pprint.PrettyPrinter(indent=indent)
    pp.pprint(text)

# Loading Parameters

In [3]:
modelID = "gpt-3.5-turbo"

# Patient symptoms analysis - Sequential Chain:

Based on the patient's input information, we want the language model to make a diagnosis and a recommendation to a practitioner.  
As the first try, we do it with a single call to the LLM.


In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model=modelID)


The chain breakdown is as follows:
1. translate the input inquiry to english.
2. Symptom Extraction Agent Prompt.
3. symptoms categorization.
4. summarization and medical recommendation.
5. Finding the language of the original inquiry.
6. Translating the summary to original language

The first chain: Translate the input inquiry to english

In [6]:
prompt1.pretty_print()

================================ Human Message =================================

Translate the following patient input from its native language to English while preserving medical terms and symptoms.If the input language is in English just give the input inquary to the output.

inquiry: {inquiry}


In [5]:
from langchain.prompts import ChatPromptTemplate
# from langchain_core.output_parsers import StrOutputParser
from langchain.chains import LLMChain

# prompt template 1: translating the input inquiry to english
prompt1 = ChatPromptTemplate.from_template(
    "Translate the following patient input from its native language to English while preserving medical terms and symptoms."
    "If the input language is in English just give the input inquary to the output."
    "\n\ninquiry: {inquiry}"
)
# chain 1: 
chain1 = LLMChain(llm=llm, 
                  prompt=prompt1, 
                  output_key="English_inquiry"
                  )



d:\Portfolio\LLMs Applications\.llmenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


The second chain: Symptom Extraction

In [8]:
prompt2 = ChatPromptTemplate.from_template(
    "Your role is to identify and extract specific medical symptoms from patient descriptions in the following inquiry."
    "Symptoms are the subjective experiences reported by the patient, such as pain, dizziness, or nausea, which indicate a potential health issue." 
    "Accurate extraction of symptoms is critical for diagnosis and treatment planning."    
    "\n\nInquiry: {English_inquiry}"
)

chain2 = LLMChain(llm=llm, 
                  prompt=prompt2, 
                  output_key="Symptoms"
                  )

The third chain: symptoms categorization

In [18]:
prompt3 = ChatPromptTemplate.from_template(
    "As a Symptom Analysis Agent, your role is to analyze the followin list of extracted symptoms and categorize them into relevant medical fields."
    "This categorization helps in identifying potential underlying conditions and determining the appropriate medical specialty for further evaluation."
    "The medical fields include, but are not limited to, neurology, infectious diseases, cardiology, orthopedics, and gastroenterology"
    "\n\nSymptoms: {Symptoms}"
)

chain3 = LLMChain(llm=llm, 
                  prompt=prompt3, 
                  output_key="categorized_symptoms"
                  )

The fourth chain: summarization and medical recommendation

In [19]:
prompt4 = ChatPromptTemplate.from_template(
    "As a Medical Summarization Agent, your task is to compile and summarize the symptoms and their categorization into a coherent, concise summary that is easily understandable by a human healthcare practitioner."
    "Your summary should accurately reflect the patient's condition, include any pertinent details from the analysis, and provide clear recommendations for the next steps." 
    "Your ultimate goal is to aid healthcare practitioners in quickly understanding the patient's situation and determining the appropriate course of action."
    "\n\nCategorized symptoms: {categorized_symptoms}"
)

chain4 = LLMChain(llm=llm, 
                  prompt=prompt4, 
                  output_key="Final_Summary"
                  )

The fifth chain: Finding the language of the original inquiry

In [53]:
prompt5 = ChatPromptTemplate.from_template(
    "Find the language of the original inquiry."    
    "\n\noriginal inquary: {inquiry}"
)

chain5 = LLMChain(llm=llm,
                  prompt=prompt5,
                  output_key="language"
                  )

The sixth chain: Translating the summary to original language

In [54]:
prompt6 = ChatPromptTemplate.from_template(
    "Translate this medical summary and recommendaton to the language of the original inquiry."    
    "\n\n summary: {Final_Summary} \n\nlanguage: {language}"
)

chain6 = LLMChain(llm=llm, 
                  prompt=prompt6, 
                  output_key="Translated_final_Summary"
                  )

Linking all chains together

In [58]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4, chain5, chain6],
    input_variables=["inquiry"],
    output_variables=["English_inquiry", "Symptoms", "categorized_symptoms", "Final_Summary", "language", "Translated_final_Summary"],
    verbose=True
)
overall_chain.output_variables

['English_inquiry',
 'Symptoms',
 'categorized_symptoms',
 'Final_Summary',
 'language',
 'Translated_final_Summary']

In [59]:
inquiry = """
Seit einigen Tagen habe ich starke Kopfschmerzen und fühle mich sehr schwach. \n
Ich habe auch hohes Fieber und Schüttelfrost. Außerdem habe ich bemerkt, dass meine Sehkraft sich verschlechtert hat und ich manchmal verschwommen sehe.\n
Meine Gelenke tun weh, besonders in den Knien und Handgelenken, und ich habe Schwierigkeiten, mich zu bewegen.\n
Ich habe auch Übelkeit und manchmal erbreche ich. Mein Appetit ist stark zurückgegangen und ich fühle mich ständig müde.\n
In letzter Zeit habe ich auch leichte Atembeschwerden bemerkt, besonders nachts.\n 
Können Sie mir helfen herauszufinden, was mit mir los ist?
"""
completion = overall_chain(inquiry)
completion



> Entering new SequentialChain chain...

> Finished chain.


In [61]:
print(completion['English_inquiry'])

For a few days now, I have been experiencing severe headaches and feeling very weak.

I also have a high fever and chills. Additionally, I have noticed that my vision has deteriorated and I sometimes see things blurry.

My joints are painful, especially in my knees and wrists, and I have difficulty moving.

I also have nausea and sometimes vomit. My appetite has significantly decreased and I constantly feel tired.

Lately, I have also noticed slight difficulty breathing, especially at night.

Can you help me figure out what is wrong with me?


In [62]:
print(completion['Symptoms'])

Symptoms identified from the patient description:
1. Severe headaches
2. Feeling very weak
3. High fever
4. Chills
5. Deteriorated vision
6. Blurry vision
7. Painful joints (especially in knees and wrists)
8. Difficulty moving
9. Nausea
10. Vomiting
11. Decreased appetite
12. Constant fatigue
13. Slight difficulty breathing (especially at night)


In [63]:
print(completion['categorized_symptoms'])

Based on the list of symptoms provided, the categorization into relevant medical fields would be as follows:

1. Severe headaches, Deteriorated vision, Blurry vision - Neurology
2. Feeling very weak, Constant fatigue - General Medicine/Internal Medicine
3. High fever, Chills, Nausea, Vomiting, Decreased appetite - Infectious Diseases
4. Painful joints (especially in knees and wrists), Difficulty moving - Orthopedics
5. Slight difficulty breathing (especially at night) - Pulmonology/Respiratory Medicine

It is important to note that some symptoms may overlap across different medical fields, and a comprehensive evaluation by a healthcare professional is necessary to determine the underlying condition and appropriate treatment plan.


In [64]:
nprint(completion['Final_Summary'])

('Summary:\n'
 '\n'
 'The patient presents with a range of symptoms spanning multiple medical '
 'fields. They are experiencing severe headaches, deteriorated vision, and '
 'blurry vision, indicating a potential neurological issue. Additionally, they '
 'report feeling very weak, constant fatigue, high fever, chills, nausea, '
 'vomiting, decreased appetite, painful joints (especially in knees and '
 'wrists), difficulty moving, and slight difficulty breathing (especially at '
 'night).\n'
 '\n'
 'Given the complexity and variety of symptoms, a thorough evaluation by '
 'healthcare professionals in neurology, general medicine/internal medicine, '
 'infectious diseases, orthopedics, and pulmonology/respiratory medicine is '
 'recommended. Further diagnostic tests and imaging may be necessary to '
 'determine the underlying condition and develop an appropriate treatment '
 "plan. Immediate medical attention is advised to address the patient's "
 'symptoms and provide necessary care.')


In [66]:
nprint(completion['language'])
nprint(completion['Translated_final_Summary'])

'German'
('Zusammenfassung:\n'
 '\n'
 'Der Patient zeigt eine Vielzahl von Symptomen, die mehrere medizinische '
 'Bereiche betreffen. Er leidet unter starken Kopfschmerzen, verschlechtertem '
 'Sehvermögen und verschwommenem Sehen, was auf ein mögliches neurologisches '
 'Problem hinweist. Darüber hinaus berichtet er von starker Schwäche, '
 'ständiger Müdigkeit, hohem Fieber, Schüttelfrost, Übelkeit, Erbrechen, '
 'verminderter Appetit, schmerzhaften Gelenken (insbesondere in den Knien und '
 'Handgelenken), Bewegungsschwierigkeiten und leichten Atembeschwerden '
 '(insbesondere nachts).\n'
 '\n'
 'Aufgrund der Komplexität und Vielfalt der Symptome wird eine gründliche '
 'Untersuchung durch Fachleute im Bereich Neurologie, Allgemeinmedizin/Innere '
 'Medizin, Infektionskrankheiten, Orthopädie und Pulmonologie/Atemwegsmedizin '
 'empfohlen. Weitere diagnostische Tests und Bildgebung können erforderlich '
 'sein, um die zugrunde liegende Erkrankung festzustellen und einen '
 'angemess

# Patient symptoms analysis - Router Chain:

Now we want to perform a symptom based chain call.  
Each llm chain is customized for one specific medical department and report the result to the relevant specialist in that department.  
For this purpose, we use the Router Chain module of Langchain.  

**Note:** This module's functionality is close to what we can do with LLM agents and tools, but at a simpler level.


## Router Chain Structure:
We will have the following department customized chains. They can summarize and give medical recommendations for the medical chief of the department.
1. Cardiology Chain
2. Neurology Chain
3. Gastroenterology Chain
4. Orthopedics Chain

Then a router chain select the appropriate department based on its understanding from the patient symptoms.

Creating chain descriptions:

In [23]:
prompt_infos = list()
departments = ["Cardiology", "Neurology", "Orthopedics", "Gastroenterology"]
Conditions = ["cardiovascular", "neurological", "orthopedic", "gastroenterological"]
for department, condition in zip(departments, Conditions):
    template = f"""
    You are a {department} specialist. A patient has reported the following symptoms.\
    Extract, and analyze the symptoms to identify potential {condition} conditions.\
    Summarize your findings and provide recommendations to the chief medical officer.\
    """
    template += """\n
    Symptoms: 
    {input}
    """

    prompt_info = {
        "name": f'{department}', 
        "description": f"Processes patient symptoms to identify and analyze potential {condition} conditions", 
        "prompt_template": template
        }    
    prompt_infos.append(prompt_info)
prompt_infos

[{'name': 'Cardiology',
  'description': 'Processes patient symptoms to identify and analyze potential cardiovascular conditions',
  'prompt_template': '\n    You are a Cardiology specialist. A patient has reported the following symptoms.    Extract, and analyze the symptoms to identify potential cardiovascular conditions.    Summarize your findings and provide recommendations to the chief medical officer.    \n\n    Symptoms: \n    {input}\n    '},
 {'name': 'Neurology',
  'description': 'Processes patient symptoms to identify and analyze potential neurological conditions',
  'prompt_template': '\n    You are a Neurology specialist. A patient has reported the following symptoms.    Extract, and analyze the symptoms to identify potential neurological conditions.    Summarize your findings and provide recommendations to the chief medical officer.    \n\n    Symptoms: \n    {input}\n    '},
 {'name': 'Orthopedics',
  'description': 'Processes patient symptoms to identify and analyze pote

Creating the list of llm chains from descriptions:

In [24]:
from langchain.prompts import ChatPromptTemplate

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

The default chain as to transfer the patient to ER for treatment:

In [35]:
template = """
You are an Emergency Physician. A patient has reported the following symptoms.\
Extract, and analyze the symptoms to identify potential conditions.\
Summarize your findings and recommend the further treatment in the Emergency department.\

Symptoms: 
{input}
"""
default_prompt = ChatPromptTemplate.from_template(template)
default_chain = LLMChain(llm=llm, prompt=default_prompt)

Creating the router chain:

In [36]:
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser


ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

router_template = ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

<>:5: SyntaxWarning: invalid escape sequence '\ '
<>:5: SyntaxWarning: invalid escape sequence '\ '
C:\Users\bbkho\AppData\Local\Temp\ipykernel_50364\4130741913.py:5: SyntaxWarning: invalid escape sequence '\ '
  ROUTER_TEMPLATE = """Given a raw text input to a \


In [41]:
from langchain.chains.router import MultiPromptChain
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

Let's test it out:

In [33]:
symptoms = """
The patient reports experiencing severe chest pain, shortness of breath, and a rapid heartbeat, especially during physical exertion.
"""
completion = chain.run(symptoms)
nprint(completion)



> Entering new MultiPromptChain chain...
Cardiology: {'input': 'The patient reports experiencing severe chest pain, shortness of breath, and a rapid heartbeat, especially during physical exertion.'}
> Finished chain.
('Based on the reported symptoms of severe chest pain, shortness of breath, '
 'and a rapid heartbeat during physical exertion, the patient may be '
 'experiencing symptoms of a potential cardiovascular condition. These '
 'symptoms are commonly associated with conditions such as angina, heart '
 'attack, or heart failure.\n'
 '\n'
 'Angina is chest pain or discomfort that occurs when the heart muscle does '
 'not receive enough oxygen-rich blood. It is often triggered by physical '
 'exertion or stress. A heart attack occurs when there is a blockage in the '
 'blood flow to the heart muscle, leading to damage or death of the heart '
 'tissue. Symptoms include severe chest pain, shortness of breath, and a rapid '
 'heartbeat. Heart failure is a condition where the heart 

In [31]:
symptoms = """
The patient complains of recurring severe headaches, dizziness, and occasional numbness in the left arm and face.
"""
completion = chain.run(symptoms)
nprint(completion)



> Entering new MultiPromptChain chain...
Neurology: {'input': 'The patient complains of recurring severe headaches, dizziness, and occasional numbness in the left arm and face.'}
> Finished chain.


'Based on the reported symptoms of recurring severe headaches, dizziness, and occasional numbness in the left arm and face, there are several potential neurological conditions that could be causing these symptoms. \n\nOne possible condition could be migraines, which are characterized by severe headaches often accompanied by dizziness and sensory symptoms such as numbness. Another potential condition could be a transient ischemic attack (TIA) or mini-stroke, which can cause temporary numbness in the face and arm along with dizziness. Additionally, conditions such as multiple sclerosis or a brain tumor could also present with these symptoms.\n\nIt is important to conduct further diagnostic tests such as imaging studies (MRI or CT scans), blood tests, and possibly a neurological examination to determine the underlying cause of these symptoms. Treatment will depend on the specific diagnosis, but may include medications for pain management, prevention of migraines, or addressing any underly

Let's try an ER case!

In [42]:
symptoms = """
The patient has been experiencing a persistent rash, itching, and swelling on the arms and legs, with no apparent trigger.
"""
completion = chain.run(symptoms)
nprint(completion)



> Entering new MultiPromptChain chain...
None: {'input': 'The patient has been experiencing a persistent rash, itching, and swelling on the arms and legs, with no apparent trigger.'}
> Finished chain.
('Potential conditions based on the symptoms reported by the patient could '
 'include allergic reactions, contact dermatitis, eczema, or even an '
 'underlying autoimmune condition such as lupus.\n'
 '\n'
 'To further investigate and confirm the diagnosis, I would recommend '
 'conducting a thorough physical examination, including a detailed medical '
 'history to identify any potential triggers or underlying health conditions. '
 'Additionally, blood tests, skin biopsies, or allergy testing may be '
 'necessary to determine the cause of the symptoms.\n'
 '\n'
 'In the meantime, symptomatic treatment can be provided in the Emergency '
 'department to alleviate the rash, itching, and swelling. This may include '
 'prescribing antihistamines, corticosteroids, or topical creams to reduce 